In [62]:
from collections.abc import Mapping
from numpy import typing as npy
from typing import cast, Any

import numpy as np
from sklearn.cross_decomposition import PLSRegression


def params(plsr: PLSRegression) -> Mapping[str, npy.NDArray[Any]]:
    return cast(Mapping[str, npy.NDArray[Any]], {'x_weights_': plsr.x_weights_, # type: ignore
                                                 'x_loadings_': plsr.x_loadings_, # type: ignore
                                                 'n_components': plsr.n_components, # type: ignore
                                                 'x_mean_': plsr._x_mean, # type: ignore
                                                 'x_std_': plsr._x_std, # type: ignore 
                                                 'scale': plsr.scale}) # type: ignore


def apply(X: npy.NDArray[Any], params: Mapping[str, npy.NDArray[Any]]):
    X = np.copy(X) # type: ignore
    X_preprocessed = X - params['x_mean_']
    if params['scale']:
        X_preprocessed /= params['x_std_']
    x_scores = np.dot(X_preprocessed, params['x_weights_'])
    for i in range(1, params['n_components']):
        x_scores[:, i] -= np.dot(x_scores[:, :i], np.dot(params['x_loadings_'][:, :i].T, params['x_weights_'][:, i]))
    return x_scores

In [64]:
from sklearn.model_selection import train_test_split


np.random.seed(0)
X = np.random.randn(100, 10)
y = 2 * X[:, 0] + 3 * X[:, 1] + np.random.randn(100) * 0.1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) # type: ignore

pls = PLSRegression(n_components=2, scale=True)
pls.fit(X_train, y_train)

pls_params = params(pls)

X_test_transformed_custom = apply(X_test, pls_params)
X_test_transformed_sklearn = pls.transform(X_test)

print("Custom transformation shape:", X_test_transformed_custom.shape)
print("Sklearn transformation shape:", X_test_transformed_sklearn.shape)
print("Are the transformations equal?", np.allclose(X_test_transformed_custom, X_test_transformed_sklearn, atol=1e-6))
print("Max absolute difference:", np.max(np.abs(X_test_transformed_custom - X_test_transformed_sklearn)))
print(X_test_transformed_custom)
print(X_test_transformed_sklearn)

Custom transformation shape: (20, 2)
Sklearn transformation shape: (20, 2)
Are the transformations equal? True
Max absolute difference: 4.440892098500626e-16
[[ 1.10054354 -0.99897029]
 [-1.05498384 -0.30875977]
 [-1.29340448  1.38815205]
 [ 0.01143077 -1.17530443]
 [-0.95761658  0.48550216]
 [ 0.81286234  1.69808121]
 [-0.72034819  0.01340318]
 [ 1.64968309 -0.81040518]
 [ 0.4197219  -0.03657242]
 [ 1.67097015 -0.42702694]
 [-0.94689159  1.54984727]
 [ 0.62744775 -0.53366962]
 [-0.59960806  1.4603698 ]
 [-0.67666534 -1.21246017]
 [-1.21986103  0.18415932]
 [-1.7115407   0.01573084]
 [ 0.20509978  0.34509984]
 [-0.10625842  0.31775087]
 [-0.47144755 -0.06369227]
 [ 1.08371974  0.03306419]]
[[ 1.10054354 -0.99897029]
 [-1.05498384 -0.30875977]
 [-1.29340448  1.38815205]
 [ 0.01143077 -1.17530443]
 [-0.95761658  0.48550216]
 [ 0.81286234  1.69808121]
 [-0.72034819  0.01340318]
 [ 1.64968309 -0.81040518]
 [ 0.4197219  -0.03657242]
 [ 1.67097015 -0.42702694]
 [-0.94689159  1.54984727]
 [ 0